In [4]:
import sqlite3
import pandas as pd
from datetime import datetime

# Connect to database
conn = sqlite3.connect('unified_messages.db')
cursor = conn.cursor()

# Get column names first
cursor.execute("PRAGMA table_info(messages)")
columns = [column[1] for column in cursor.fetchall()]

# Fetch recent messages
cursor.execute("SELECT * FROM messages ORDER BY saved_at DESC LIMIT 10")
recent_messages = cursor.fetchall()

# Convert to DataFrame for better handling
df = pd.DataFrame(recent_messages, columns=columns)

# Filter out None values and display
print("=== Last 10 Recent Messages (Filtered) ===")
print(f"Total messages in database: {len(df)}")

# Display key columns with None filtering
key_columns = ['id', 'source', 'content', 'author', 'timestamp', 'sentiment', 'category', 'subreddit', 'channel_name']
filtered_df = df[key_columns].copy()

# Replace None with 'N/A' for better display
filtered_df = filtered_df.fillna('N/A')

# Truncate long content for better readability
filtered_df['content'] = filtered_df['content'].str[:80] + '...'
filtered_df['id'] = filtered_df['id'].str[:20] + '...'

print("\nFiltered and formatted data:")
print(filtered_df.to_string(index=False))

# Show statistics
print(f"\n=== Statistics ===")
print(f"Sources: {df['source'].value_counts().to_dict()}")
print(f"Sentiments: {df['sentiment'].value_counts().to_dict()}")
print(f"Categories: {df['category'].value_counts().to_dict()}")

conn.close()

=== Last 10 Recent Messages (Filtered) ===
Total messages in database: 10

Filtered and formatted data:
                     id source                                   content      author                  timestamp sentiment        category subreddit channel_name
slack_general_176047...  slack                      App keep crashing... U09L2J0Q153 2025-10-15T02:20:25.100079   neutral             bug       N/A          N/A
slack_general_176047...  slack  Facing issue due to bad ui experience... U09L2J0Q153 2025-10-15T02:19:20.005649  negative             bug       N/A          N/A
slack_general_176047...  slack   You need to change the ui of the app... U09L2J0Q153 2025-10-15T02:18:57.188849   neutral feature_request       N/A          N/A
slack_general_176047...  slack                        Cashout problem... U09L2J0Q153 2025-10-15T02:17:31.376909   neutral             bug       N/A          N/A
slack_general_176047...  slack                          Bad ui design... U09L2J0Q153 2025-1

In [ ]:
# Advanced filtering options
import sqlite3
import pandas as pd

def get_filtered_messages(limit=10, source=None, sentiment=None, category=None, exclude_none=True):
    """
    Get filtered messages from database
    
    Args:
        limit: Number of messages to return
        source: Filter by source ('slack', 'reddit', etc.)
        sentiment: Filter by sentiment ('positive', 'negative', 'neutral')
        category: Filter by category ('bug', 'feature_request', etc.)
        exclude_none: Whether to exclude None values
    """
    conn = sqlite3.connect('unified_messages.db')
    
    # Build query
    query = "SELECT * FROM messages WHERE 1=1"
    params = []
    
    if source:
        query += " AND source = ?"
        params.append(source)
    
    if sentiment:
        query += " AND sentiment = ?"
        params.append(sentiment)
        
    if category:
        query += " AND category = ?"
        params.append(category)
    
    if exclude_none:
        query += " AND content IS NOT NULL AND content != ''"
    
    query += " ORDER BY saved_at DESC LIMIT ?"
    params.append(limit)
    
    # Execute query
    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    
    return df

# Example usage
print("=== Filtered Messages Examples ===")

# Get only Slack messages
slack_messages = get_filtered_messages(limit=5, source='slack')
print(f"\nLast 5 Slack messages:")
print(slack_messages[['id', 'content', 'sentiment', 'category']].fillna('N/A').to_string(index=False))

# Get only negative sentiment messages
negative_messages = get_filtered_messages(limit=5, sentiment='negative')
print(f"\nLast 5 negative messages:")
print(negative_messages[['id', 'content', 'source', 'category']].fillna('N/A').to_string(index=False))

# Get only bug reports
bug_messages = get_filtered_messages(limit=5, category='bug')
print(f"\nLast 5 bug reports:")
print(bug_messages[['id', 'content', 'source', 'sentiment']].fillna('N/A').to_string(index=False))


=== Filtered Messages Examples ===

Last 5 Slack messages:
                                         id                               content sentiment        category
slack_general_1760475025.100079_U09L2J0Q153                     App keep crashing   neutral             bug
slack_general_1760474960.005649_U09L2J0Q153 Facing issue due to bad ui experience  negative             bug
slack_general_1760474937.188849_U09L2J0Q153  You need to change the ui of the app   neutral feature_request
slack_general_1760474851.376909_U09L2J0Q153                       Cashout problem   neutral             bug
slack_general_1760474844.305949_U09L2J0Q153                         Bad ui design  negative           other

Last 5 negative messages:
                                         id                                                                                                                                                                                                                               

: 

In [ ]:
# Database Statistics and Summary
import sqlite3
import pandas as pd

def get_database_summary():
    """Get comprehensive database statistics"""
    conn = sqlite3.connect('unified_messages.db')
    
    # Basic counts
    total_messages = pd.read_sql_query("SELECT COUNT(*) as count FROM messages", conn).iloc[0]['count']
    
    # Source breakdown
    source_stats = pd.read_sql_query("""
        SELECT source, COUNT(*) as count 
        FROM messages 
        GROUP BY source 
        ORDER BY count DESC
    """, conn)
    
    # Sentiment breakdown
    sentiment_stats = pd.read_sql_query("""
        SELECT sentiment, COUNT(*) as count 
        FROM messages 
        WHERE sentiment IS NOT NULL
        GROUP BY sentiment 
        ORDER BY count DESC
    """, conn)
    
    # Category breakdown
    category_stats = pd.read_sql_query("""
        SELECT category, COUNT(*) as count 
        FROM messages 
        WHERE category IS NOT NULL
        GROUP BY category 
        ORDER BY count DESC
    """, conn)
    
    # Recent activity (last 24 hours)
    recent_activity = pd.read_sql_query("""
        SELECT COUNT(*) as count 
        FROM messages 
        WHERE saved_at > datetime('now', '-1 day')
    """, conn).iloc[0]['count']
    
    # Top subreddits/channels
    top_channels = pd.read_sql_query("""
        SELECT 
            COALESCE(subreddit, channel_name, 'Unknown') as channel,
            COUNT(*) as count
        FROM messages 
        WHERE COALESCE(subreddit, channel_name) IS NOT NULL
        GROUP BY COALESCE(subreddit, channel_name)
        ORDER BY count DESC
        LIMIT 10
    """, conn)
    
    conn.close()
    
    return {
        'total_messages': total_messages,
        'recent_activity': recent_activity,
        'source_stats': source_stats,
        'sentiment_stats': sentiment_stats,
        'category_stats': category_stats,
        'top_channels': top_channels
    }

# Get and display summary
summary = get_database_summary()

print("=== DATABASE SUMMARY ===")
print(f"📊 Total Messages: {summary['total_messages']:,}")
print(f"🕐 Recent Activity (24h): {summary['recent_activity']:,}")
print(f"📈 Processing Rate: {(summary['recent_activity']/summary['total_messages']*100):.1f}% recent")

print(f"\n📱 Sources:")
for _, row in summary['source_stats'].iterrows():
    print(f"   {row['source']}: {row['count']:,}")

print(f"\n😊 Sentiments:")
for _, row in summary['sentiment_stats'].iterrows():
    print(f"   {row['sentiment']}: {row['count']:,}")

print(f"\n🏷️ Categories:")
for _, row in summary['category_stats'].iterrows():
    print(f"   {row['category']}: {row['count']:,}")

print(f"\n📺 Top Channels/Subreddits:")
for _, row in summary['top_channels'].iterrows():
    print(f"   {row['channel']}: {row['count']:,}")
